<a href="https://colab.research.google.com/github/WyattVO/PytorchBasics/blob/main/pytorchMNIST_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# data->model->loss->optimizer->backdrop->update.


import torch #pytorch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn

# 1. Load MNIST dataset
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(
    root="./data", #where to store data.
    train=True,
    transform=transform,
    download=True
)
#Test datset for after training.
test_dataset = datasets.MNIST(
    root="./data",
    train=False,    # <-- test set instead of train
    transform=transform,
    download=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
#Batch sizes at 64 because its a power of 2.


# 2. Define a simple model: Linear -> ReLU -> Linear
# MNIST images are 1 x 28 x 28, so input size is 28*28 = 784
model = nn.Sequential(
    nn.Flatten(),           # [B,1,28,28] -> [B,784] Images are 28x28 then flattening to 28x28. 1D vector.
    nn.Linear(784, 128), #Each pixel to a neuron.
    nn.ReLU(), #Rectified Linear Unit. This is bascially x<0 goes to 0.
    nn.Linear(128, 10)      # 128 neurons to 10 outputs. MNIST 0-9. logits to classify loss.
)

#This is going to use GPU using the .to function.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Loss and optimizer
loss_fn = nn.CrossEntropyLoss()#logits into probability/penalizes wrong outcomes.
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
#moves weight by how much for each wrong/right. 0.05 was better than 0.1.

#Training it under 5 different datasets. Trying to get higher than 90%
num_epochs = 5
# 4. Training loop
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (images, labels) in enumerate(train_loader):
    #Before forward pass now that im trying to move the model to GPU use .to on images and labels.
      images = images.to(device)
      labels = labels.to(device)
      # forward pass
      logits = model(images)              # shape [B,10] from the prior setup.
      loss = loss_fn(logits, labels)

      # zero gradients
      optimizer.zero_grad()
      # backward pass
      loss.backward()#reflects on outcome.

      # step
      optimizer.step()




    # print loss every 200 batches
      if batch_idx % 200 == 0:
          print(f"Batch {batch_idx:04d}  Loss: {loss.item():.4f}")

#DONE TRAINING
model.eval()  # switch to evaluation mode
correct = 0
total = 0
# needs test dataset compared to trained data set.
with torch.no_grad():  # disables gradient tracking for speed
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")




Batch 0000  Loss: 2.3005
Batch 0200  Loss: 0.4989
Batch 0400  Loss: 0.3016
Batch 0600  Loss: 0.3300
Batch 0800  Loss: 0.2506
Batch 0000  Loss: 0.4580
Batch 0200  Loss: 0.4551
Batch 0400  Loss: 0.1230
Batch 0600  Loss: 0.2454
Batch 0800  Loss: 0.3089
Batch 0000  Loss: 0.2162
Batch 0200  Loss: 0.2458
Batch 0400  Loss: 0.2140
Batch 0600  Loss: 0.1119
Batch 0800  Loss: 0.3912
Batch 0000  Loss: 0.1469
Batch 0200  Loss: 0.1492
Batch 0400  Loss: 0.1948
Batch 0600  Loss: 0.1639
Batch 0800  Loss: 0.1393
Batch 0000  Loss: 0.0911
Batch 0200  Loss: 0.1783
Batch 0400  Loss: 0.2734
Batch 0600  Loss: 0.3442
Batch 0800  Loss: 0.1194
Accuracy: 95.13%
